# Final Project 

Summary |
In this project I will be taking marcro economic data and corrolating that back to a investment portfolio. This portfolio includes stocks, bonds, and etfs. 

In [11]:
#plots.py
import os
import pandas
import pandas_datareader.data as web
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms
import datetime
import math

def gather_data(data_codes, start, end = datetime.datetime.today(), freq = "A"):
    i = 0
    # dct.items() calls key and value that key points to
    for key, val in data_codes.items():
        if i == 0:
            # Create dataframe for first variable, then rename column
            df = web.DataReader(val, "fred", start, end).resample(freq).mean()
            df.rename(columns = {val:key}, inplace = True) 
            # setting i to None will cause the next block of code to execute,
            # placing data within df instead of creating a new dataframe for
            # each variable
            i = None
        else:
            # If dataframe already exists, add new column
            df[key] = web.DataReader(val, "fred", start, end).resample(freq).mean()

    return df

def plot_ts_scatter(df, s = 75, figsize = (40, 20), save_fig = False, pp = None):
    # Create plot for every unique pair of variables
    plot_vars = list(df.keys())
    for var1 in plot_vars:
        for var2 in plot_vars:
            if var1 != var2:
                fig, ax = plt.subplots(figsize = figsize)
                # Create list of years from index
                # Year will be represented by color
                if "Year" not in df.keys():
                    df["Year"] = [int(str(ind)[:4]) for ind in df.index] 
                df.plot.scatter(x = var1, y = var2, s = s, ax = ax, 
                                c = "Year", cmap = "viridis")
                # Turn the text on the x-axis so that it reads vertically
                ax.tick_params(axis='x', rotation=90)
                # Get rid of tick lines perpendicular to the axis for aesthetic
                ax.tick_params('both', length=0, which='both')
                # save image if PdfPages object was passed
                if save_fig:
                    try:
                        os.mkdir("plots")
                    except:
                        pass
                    plt.savefig("plots/" + str(plot_vars).replace("[", "").replace("]","")[:40] + " scatter.png",
                            bbox_inches = "tight")
                    if pp != None: pp.savefig(fig, bbox_inches = "tight")

def plot_lines(df, title = False, linewidth = 1, figsize = (40,20), full_index = False, 
               h_line = False, max_y = False, legend = True, pp = None, show_inversion = False):
    fig, ax = plt.subplots(figsize = figsize)
    # If no secondary_y (axis), plot all variables at once
    df.plot.line(linewidth = linewidth, ax = ax, legend = legend)
    if h_line != False:
        ax.axhline(h_line, ls = "--", linewidth = 1.5, color = "k")
    # Turn the text on the x-axis so that it reads vertically
    ax.tick_params(axis='x', rotation=90)
    # Get rid of tick lines perpendicular to the axis for aesthetic
    ax.tick_params('both', length=0, which='both')
    if max_y != False:
        ax.set_ylim(bottom = 0, top = max_y)
    if full_index:
        plt.xticks([i for i in range(len(df.index))], list(df.index))
    vals = ax.get_yticks()
    ax.set_yticklabels([round(y,2) for y in vals]) 

    # transform y-axis values from sci notation to integers
    vals = ax.get_yticks()
    ax.set_yticklabels([round(y,2) for y in vals]) 
    if title != False:
        plt.title(title, fontsize = 72)

    if show_inversion:
        trans = mtransforms.blended_transform_factory(ax.transData, ax.transAxes)
        ax.fill_between(df.index, 0, df.max().max(), where=df["2 Y (%)"] < df["1 M (%)"],
                    facecolor='red', alpha=0.2, transform = trans)
    # format image filename 
    remove_chars = "[]:$'\\"
    filename = str(list(df.keys()))
    for char in remove_chars:
        filename = filename.replace(char, "")
    plt.savefig(filename[:50] + " line.png",
                bbox_inches = "tight")
    #[:50] + " line.png"
    # save image if PdfPages object was passed
    if pp != None: pp.savefig(fig, bbox_inches = "tight")

def plot_stacked_lines(df, plot_vars, linewidth = 1, figsize = (40,20), 
                       pp = None, total_var = False):
    fig, ax = plt.subplots(figsize = figsize)
#    mpl_colors = ["C" + str(i) for i in range(11)]
    df[plot_vars].plot.area(stacked = True, linewidth = linewidth,
                            ax = ax)
    if total_var != False:
        df[total_var].plot.line(linewidth = linewidth, ax = ax, c = "k",
              label = total_var, ls = "--")
    ax.legend(loc=2, ncol = 2)

In [12]:
# This is my reference code to configure all the shortcuts of my data. 

data_codes  = {"Base: Total ($ Mil)": "BOGMBASEW",
               "Base: Currency in Circulation ($ Mil)": "MBCURRCIRW",
              
              # Assets
               "Balance Sheet: Total Assets ($ Mil)": "WALCL",
               "Balance Sheet Securities, Prem-Disc, Repos, and Loans ($ Mil)": "WSRLL",
               "Balance Sheet: Securities Held Outright ($ Mil)": "WSHOSHO",
              
              ### breakdown of securities holdings ###
               "Balance Sheet: U.S. Treasuries Held Outright ($ Mil)":"WSHOTSL",
               "Balance Sheet: Federal Agency Debt Securities ($ Mil)" : "WSHOFADSL",
               "Balance Sheet: Mortgage-Backed Securities ($ Mil)": "WSHOMCB",
              
              # other forms of lending
               "Balance Sheet: Repos ($ Mil)": "WORAL",
               "Balance Sheet: Central Bank Liquidity Swaps ($ Mil)" : "SWPT",
               "Balance Sheet: Direct Lending ($ Mil)" : "WLCFLL",

              # unamortized value of securities held 
               "Balance Sheet: Unamortized Security Premiums ($ Mil)": "WUPSHO",

              # Liabilities
               "Balance Sheet: Total Liabilities ($ Mil)" : "WLTLECL",
               "Balance Sheet: Federal Reserve Notes Outstanding ($ Mil)" : "WLFN",
               "Balance Sheet: Reverse Repos ($ Mil)": "WLRRAL",
               
              # Reserves 
               "Balance Sheet: Excess Reserves ($ Mil)": "EXCSRESNW",
               "Balance Sheet: Required Reserves ($ Mil)": "RESBALREQW",
               "Balance Sheet: Total Reserves ($ Mil)": "WRESBAL",
               
              ## Major share of depsotis
               "Balance Sheet: Deposits from Dep. Institutions ($ Mil)":"WLODLL",
               "Balance Sheet: U.S. Treasury General Account ($ Mil)": "WDTGAL",
               "Balance Sheet: Other Deposits ($ Mil)": "WOTHLB",
               "Balance Sheet: All Deposits ($ Mil)": "WLDLCL",
              
               # Interest Rates
               "Federal Funds Target (Pre-Crisis)": "DFEDTAR",
               "Federal Funds (Upper) Target":"DFEDTARU",
               "Effective Federal Funds Rate":"DFF",
               "Interest on Excess Reserves":"IOER",
              
               # Req Reserves and Vault Cash
               "Vault Cash ($ Mil)": "TLVAULTW",
               "Vault Cash Used as Req. ($ Mil)": "VAULT",
              
               # Interest Rates
               "IOER (%)":"IOER",
               "EFFR (%)": "DFF",
               "1 M (%)": "DGS1MO",
               "3 M (%)": "DGS3MO",
               "6 M (%)": "DGS6MO",               
               "1 Y (%)": "DGS1",
               "2 Y (%)": "DGS2",
               "3 Y (%)": "DGS3",
               "5 Y (%)": "DGS5",
               "7 Y (%)": "DGS7",
               "10 Y (%)": "DGS10",
               "20 Y (%)": "DGS20",
               "30 Y (%)": "DGS30",
               "Bank Prime Loan Rate (%)" : "MPRIME",
               
              # Inflation 
              "Expected Inflation (%)":"T10YIE",
              "Inflation (%)": "CPIAUCSL",
              "Consumer Price Index for All Urban Consumers" : "CPIOGSNS",
              "Consumer Price Index for All Urban Consumers: All Items Less Food and Energy in U.S. City Average" : "CPILFENS",
              "Producer Price Index by Commodity for Finished Goods Less Food and Energy" : "PPILFE",
              
               # Work Week
               "Manufacturing Workweek (Hours)": "AWHMAN",
               "Construction Workweek (Hours)": "AWHAECON",
               
               # Consumer Sentiment
               "Consumer Sentiment (Index)" : "UMCSENT",
               "Personal Income" : "PI",
               "Personal current transfer payments" : "W211RC1",
               "Real Disposable Personal Income ($B)" : "DSPIC96",
               "Nonfinancial Corporate Business: Profits After Tax ($B)" : "NFCPATAX",
    
               # Durable Goods
               "Durable Goods Orders (millions)" : "DGORDER",
               "Personal Consumption Expenditures (Index)" : "PCECTPI",
    
               #Financial Conditions
               "Chicago Fed National Financial Conditions Index (Index)" : "NFCINONFINLEVERAGE",
               "S&P 500 Stock Price Index ($)" : "SP500",
               "recessions" : "JHDUSRGDPBR",
               "Corporate Profits After Tax ($B)" : "CP",
               
               # Employment 
               "4-Week Moving Average of Initial Claims" : "IC4WSA",
               "Median Weeks Unemployed (Weeks)" : "UEMPMED",
               "Labor Force Participation Rate (%)" : "CIVPART",
               "Total Unemployed, Plus All Persons Marginally Attached to the Labor Force, Plus Total Employed Part Time for Economic Reasons, as a Percent of the Civilian Labor Force Plus All Persons Marginally Attached to the Labor Force (%)" : "U6RATE",
               "Unemployment Rate (%)" : "UNRATE",
               "Hires: Total Nonfarm" : "JTSHIL",
               "All Employees, Total Nonfarm (Thousands of Persons)" : "PAYEMS",
               "Compensation of Employees, Received: Wage and Salary Disbursements ($B)" : "A576RC1",
               
               # New Construction Progress
               "New Private Housing Units Authorized by Building Permits" : "PERMIT",
               "Industrial Production: Total Index" : "INDPRO",
               "Manufacturers New Orders:" : "NEWORDER",
               
               # GDP
               "Real Gross Domestic Product ($B)" : "GDPC1",
               "Chicago Fed National Activity Index" : "CFNAI",
               
               # Commodity Prices 
               "US All Grades All Formulations Gas Price ($ per Gallon)" : "GASALLM",
               "Spot Oil Price: West Texas Intermediate ($ per Barrel)" : "OILPRICE",
               
               # Miscellaneous
               "Retail Sales: Retail and Food Services (Millions ($))" : "RSAFS",
               "Commercial and Industrial Loans, All Commercial Banks ($B)" : "BUSLOANS",
               "Total Consumer Credit Owned and Securitized, Outstanding ($B)" : "TOTALSL",
               "Total Business: Inventories to Sales Ratio" : "ISRATIO",
               "Total Consumer Credit Owned and Securitized, Outstanding ($B)" : "TOTALNS",
               "Business Sector: Unit Labor Cost" : "ULCBS",               
               "Federal Debt: Total Public Debt ($M)" : "GFDEBTN",
               "Households and Nonprofit Organizations; Debt Securities and Loans; Liability, Level ($B)" : "CMDEBT",
               "M2 Money Stock ($B)" : "M2",
               "Velocity of M2 Money Stock" : "M2V"}
               
# Select start and end dates
start = datetime.datetime(2006, 1, 1)
end = datetime.datetime.today()

# freq refers to data frequency. Choose "D", "W", "M", "Q", "A"
# a number may also be place in front of a letter. "2D" indicates
#       alternating days
data = gather_data(data_codes = data_codes, start = start, end = end, freq = "M")

In [13]:
data

Base: Total ($ Mil)  Base: Currency in Circulation ($ Mil)  \
DATE                                                                     
2006-01-31            805497.25                              786495.25   
2006-02-28            803651.75                              785372.75   
2006-03-31            809830.20                              789024.60   
2006-04-30            808789.00                              790543.50   
2006-05-31            812418.20                              794165.80   
...                         ...                                    ...   
2020-05-31           5139460.00                             1928941.25   
2020-06-30           5059368.25                             1955540.75   
2020-07-31           4725875.20                             1977793.40   
2020-08-31           4790990.00                             2004558.50   
2020-09-30           4876690.00                             2022000.00   

            Balance Sheet: Total Assets ($ Mil)  \
DATE                                              
2006-01-31                            836539.00   
2006-02-28                            833804.00   
2006-03-31                            838655.60   
2006-04-30                            838093.75   
2006-05-31                            842451.20   
...                                         ...   
2020-05-31                           6947555.25   
2020-06-30                           7127786.25   
2020-07-31                           6960429.40   
2020-08-31                           6975892.25   
2020-09-30                           7048374.20   

            Balance Sheet Securities, Prem-Disc, Repos, and Loans ($ Mil)  \
DATE                                                                        
2006-01-31                                           781465.0               
2006-02-28                                           781704.5               
2006-03-31                                           788354.8               
2006-04-30                                           786971.5               
2006-05-31                                           790652.4               
...                                                       ...               
2020-05-31                                          6413832.0               
2020-06-30                                          6582068.0               
2020-07-31                                          6606989.6               
2020-08-31                                          6685136.0               
2020-09-30                                          6802569.0               

            Balance Sheet: Securities Held Outright ($ Mil)  \
DATE                                                          
2006-01-31                                        744905.50   
2006-02-28                                        750386.00   
2006-03-31                                        756651.40   
2006-04-30                                        758843.00   
2006-05-31                                        760776.20   
...                                                     ...   
2020-05-31                                       5843195.25   
2020-06-30                                       6048567.25   
2020-07-31                                       6188826.40   
2020-08-31                                       6283343.50   
2020-09-30                                       6397502.00   

            Balance Sheet: U.S. Treasuries Held Outright ($ Mil)  \
DATE                                                               
2006-01-31                                          744905.50      
2006-02-28                                          750386.00      
2006-03-31                                          756651.40      
2006-04-30                                          758843.00      
2006-05-31                                          760776.20      
...                                                       ...      
2020-05-31         